<a href="https://colab.research.google.com/github/kxtubh/Deep-Learning/blob/main/bark_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-7jmphc2e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-7jmphc2e
  Resolved https://github.com/huggingface/transformers.git to commit 953196a43dae6a3c474165fba7d215fcbc7b7730
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11132583 sha256=bfc0bacd797a24b48b025f77cca29bb9a2ab17a536b98a79ae3ce19e5d52b73c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ros1882m/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3


!pip install git+https://github.com/suno-ai/bark.git

In [ ]:
from transformers import AutoProcessor, BarkModel

processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

speaker_embeddings_path.json:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

ModuleNotFoundError: No module named 'bark'

In [ ]:
preload_models()

In [ ]:
from bark.generation import preload_models, _load_model, _get_ckpt_path
import torch
import numpy as np

def my_load_model(use_gpu=False, use_small=False, force_reload=False, model_type="text"):
    """Loads a model and caches it."""
    return _load_model(use_gpu=use_gpu, use_small=use_small, force_reload=force_reload, model_type=model_type, safe_globals=[np.core.multiarray.scalar])


def preload_models(
    text_use_gpu: bool = False,
    text_use_small: bool = False,
    coarse_use_gpu: bool = False,
    coarse_use_small: bool = False,
    fine_use_gpu: bool = False,
    fine_use_small: bool = False,
    codec_use_gpu: bool = False,
    force_reload: bool = False,
) -> None:
    """
    Preloads the models into memory to avoid delays during inference.
    """
    if torch.cuda.is_available() and (
        text_use_gpu or coarse_use_gpu or fine_use_gpu or codec_use_gpu
    ):
        if torch.cuda.device_count() == 0:
            logger.warning("No GPU being used. Careful, inference might be very slow!")
        else:
            logger.info(f"Using GPU {torch.cuda.current_device()}.")
    else:
        logger.warning("No GPU being used. Careful, inference might be very slow!")
    _ = my_load_model( # calling the modified function to load the models
        model_type="text",
        use_gpu=text_use_gpu,
        use_small=text_use_small,
        force_reload=force_reload,
    )

In [ ]:
script = """
Hey, have you heard about this new text-to-audio model called "Bark"?
Apparently, it's the most realistic and natural-sounding text-to-audio model
out there right now. People are saying it sounds just like a real person speaking.
I think it uses advanced machine learning algorithms to analyze and understand the
nuances of human speech, and then replicates those nuances in its own speech output.
It's pretty impressive, and I bet it could be used for things like audiobooks or podcasts.
In fact, I heard that some publishers are already starting to use Bark to create audiobooks.
It would be like having your own personal voiceover artist. I really think Bark is going to
be a game-changer in the world of text-to-audio technology.
""".replace("\n", " ").strip()

In [ ]:
import nltk
nltk.download('punkt_tab')

sentences = nltk.sent_tokenize(script)

In [ ]:
sentences

In [ ]:
SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    audio_array = generate_audio(sentence, history_prompt=SPEAKER)
    pieces += [audio_array, silence.copy()]

In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)

In [ ]:
voice_preset = "v2/en_speaker_6"

In [ ]:
lyrics = " "

In [ ]:
inputs = processor(lyrics, voice_preset=voice_preset)

In [ ]:
audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()

In [ ]:
from IPython.display import Audio

sample_rate = model.generation_config.sample_rate
Audio(audio_array, rate=sample_rate)